<a href="https://colab.research.google.com/github/ucalyptus/wandb-experiments/blob/master/wandb_ai_Indian_Pines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# WandB – Install the W&B library
!pip install wandb -q

     |████████████████████████████████| 1.4MB 2.8MB/s 
     |████████████████████████████████| 102kB 9.2MB/s 
     |████████████████████████████████| 112kB 17.8MB/s 
     |████████████████████████████████| 102kB 9.5MB/s 
     |████████████████████████████████| 163kB 17.6MB/s 
     |████████████████████████████████| 71kB 7.6MB/s 
     |████████████████████████████████| 71kB 7.8MB/s 


In [ ]:
!pip install kornia==0.2.0

     |████████████████████████████████| 143kB 2.7MB/s 


In [ ]:
from __future__ import print_function
import argparse
import random # to set the python random seed

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import sys
import os
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from scipy import io 
import torch.utils.data
import scipy
from scipy.stats import entropy
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import math
from sklearn.metrics import mean_squared_error

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Ignore excessive warnings
import logging
logging.propagate = False 
logging.getLogger().setLevel(logging.ERROR)

# WandB – Import the wandb library
import wandb

In [ ]:
!pip install -U spectral


if not (os.path.isfile('/content/Indian_pines_corrected.mat')):
  !wget http://www.ehu.eus/ccwintco/uploads/6/67/Indian_pines_corrected.mat
if not (os.path.isfile('/content/Indian_pines_gt.mat')):
  !wget http://www.ehu.eus/ccwintco/uploads/c/c4/Indian_pines_gt.mat

     |████████████████████████████████| 194kB 2.8MB/s 
--2020-07-15 06:29:40--  http://www.ehu.eus/ccwintco/uploads/6/67/Indian_pines_corrected.mat
Resolving www.ehu.eus (www.ehu.eus)... 158.227.0.65, 2001:720:1410::65
Connecting to www.ehu.eus (www.ehu.eus)|158.227.0.65|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5953527 (5.7M) [text/plain]
Saving to: ‘Indian_pines_corrected.mat’

Indian_pines_correc 100%[===================>]   5.68M   342KB/s    in 19s     

2020-07-15 06:30:01 (305 KB/s) - ‘Indian_pines_corrected.mat’ saved [5953527/5953527]

--2020-07-15 06:30:05--  http://www.ehu.eus/ccwintco/uploads/c/c4/Indian_pines_gt.mat
Resolving www.ehu.eus (www.ehu.eus)... 158.227.0.65, 2001:720:1410::65
Connecting to www.ehu.eus (www.ehu.eus)|158.227.0.65|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1125 (1.1K) [text/plain]
Saving to: ‘Indian_pines_gt.mat’

Indian_pines_gt.mat 100%[===================>]   1.10K  --.-KB/s    in 0s   

In [ ]:
from torch.nn import Module, Sequential, Conv2d, ReLU,AdaptiveMaxPool2d, AdaptiveAvgPool2d, \
    NLLLoss, BCELoss, CrossEntropyLoss, AvgPool2d, MaxPool2d, Parameter, Linear, Sigmoid, Softmax, Dropout, Embedding


In [ ]:
# WandB – Login to your wandb account so you can log all your metrics
!wandb login

wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: 2
wandb: You chose 'Use an existing W&B account'
wandb: You can find your API key in your browser here: https://app.wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: You can find your API key in your browser here: https://app.wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: c30ac8a9c1d4811a005d8142d26b7352c536db10
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
Successfully logged in to Weights & Biases!


In [ ]:
import scipy.io as sio
def loadData():
    data = sio.loadmat('Indian_pines_corrected.mat')['indian_pines_corrected']
    labels = sio.loadmat('Indian_pines_gt.mat')['indian_pines_gt']
    
    return data, labels

In [ ]:
def padWithZeros(X, margin=2):

    ## From: https://github.com/gokriznastic/HybridSN/blob/master/Hybrid-Spectral-Net.ipynb
    newX = np.zeros((X.shape[0] + 2 * margin, X.shape[1] + 2* margin, X.shape[2]))
    x_offset = margin
    y_offset = margin
    newX[x_offset:X.shape[0] + x_offset, y_offset:X.shape[1] + y_offset, :] = X
    return newX

def createImageCubes(X, y, windowSize=5, removeZeroLabels = True):

     ## From: https://github.com/gokriznastic/HybridSN/blob/master/Hybrid-Spectral-Net.ipynb
    margin = int((windowSize - 1) / 2)
    zeroPaddedX = padWithZeros(X, margin=margin)
    # split patches
    patchesData = np.zeros((X.shape[0] * X.shape[1], windowSize, windowSize, X.shape[2]), dtype=np.uint8)
    patchesLabels = np.zeros((X.shape[0] * X.shape[1]), dtype=np.uint8)
    patchIndex = 0
    for r in range(margin, zeroPaddedX.shape[0] - margin):
        for c in range(margin, zeroPaddedX.shape[1] - margin):
            patch = zeroPaddedX[r - margin:r + margin + 1, c - margin:c + margin + 1]   
            patchesData[patchIndex, :, :, :] = patch
            patchesLabels[patchIndex] = y[r-margin, c-margin]
            patchIndex = patchIndex + 1
    if removeZeroLabels:
        patchesData = patchesData[patchesLabels>0,:,:,:]
        patchesLabels = patchesLabels[patchesLabels>0]
        patchesLabels -= 1
    return patchesData, patchesLabels


In [ ]:
class HyperSpectralDataset(Dataset):
    """HyperSpectral dataset."""

    def __init__(self,data_url,label_url,windowsize=7):
        self.windowsize=windowsize
        self.data = np.array(scipy.io.loadmat('/content/'+data_url.split('/')[-1])[data_url.split('/')[-1].split('.')[0].lower()])
        self.targets = np.array(scipy.io.loadmat('/content/'+label_url.split('/')[-1])[label_url.split('/')[-1].split('.')[0].lower()])
        self.data, self.targets = createImageCubes(self.data,self.targets, windowSize=self.windowsize)
        
        
        self.data = torch.Tensor(self.data)
        self.data = self.data.permute(0,3,1,2)
        print(self.data.shape)
        print(self.targets.shape)
        

    def __len__(self):
        return self.data.shape[0]
    
    def __getitem__(self, idx):
      
      return self.data[idx,:,:,:] , self.targets[idx]


In [ ]:
hyperparameter_defaults = dict(
    batch_size = 16,
    learning_rate = 0.01,
    momentum=0.9,
    epochs = 50,
    top=25,
    windowsize=7,
    bands=200,
    )


In [ ]:
wandb.init(config=hyperparameter_defaults,project="indian-pines")
config = wandb.config

In [ ]:
data_train = HyperSpectralDataset('Indian_pines_corrected.mat','Indian_pines_gt.mat',config.windowsize)

train_loader = DataLoader(data_train, batch_size=config.batch_size, shuffle=True)

torch.Size([10249, 200, 7, 7])
(10249,)


In [ ]:
print(data_train.__getitem__(0)[0].shape)
print(data_train.__len__())

torch.Size([200, 7, 7])
10249


In [ ]:
class PAM_Module(Module):
    """ Position attention module  https://github.com/junfu1115/DANet/blob/master/encoding/nn/attention.py"""
    #Ref from SAGAN
    def __init__(self, in_dim):
        super(PAM_Module, self).__init__()
        self.chanel_in = in_dim

        self.query_conv = Conv2d(in_channels=in_dim, out_channels=in_dim//8, kernel_size=1)
        self.key_conv = Conv2d(in_channels=in_dim, out_channels=in_dim//8, kernel_size=1)
        self.value_conv = Conv2d(in_channels=in_dim, out_channels=in_dim, kernel_size=1)
        
        self.gamma = Parameter(torch.zeros(1))

        self.softmax = Softmax(dim=-1)
    def forward(self, x):
        """
            inputs :
                x : input feature maps( B X C X H X W)
            returns :
                out : attention value + input feature
                attention: B X (HxW) X (HxW)
        """
        m_batchsize, C, height, width = x.size()
        proj_query = self.query_conv(x).view(m_batchsize, -1, width*height).permute(0, 2, 1)
        proj_key = self.key_conv(x).view(m_batchsize, -1, width*height)
        energy = torch.bmm(proj_query, proj_key)
        attention = self.softmax(energy)
        proj_value = self.value_conv(x).view(m_batchsize, -1, width*height)

        out = torch.bmm(proj_value, attention.permute(0, 2, 1))
        out = out.view(m_batchsize, C, height, width)

        out = self.gamma*out + x
        #out = F.avg_pool2d(out, out.size()[2:4])
        
        return out


class CAM_Module(Module):
    """ Channel attention module https://github.com/junfu1115/DANet/blob/master/encoding/nn/attention.py"""
    def __init__(self):
        super(CAM_Module, self).__init__()
        #self.chanel_in = in_dim
        
        


        self.gamma = Parameter(torch.zeros(1))
        self.softmax  = Softmax(dim=-1)
    def forward(self,x):
        """
            inputs :
                x : input feature maps( B X C X H X W)
            returns :
                out : attention value + input feature
                attention: B X C X C
        """
        m_batchsize, C, height, width = x.size()
        proj_query = x.view(m_batchsize, C, -1)
        proj_key = x.view(m_batchsize, C, -1).permute(0, 2, 1)
        energy = torch.bmm(proj_query, proj_key)
        energy_new = torch.max(energy, -1, keepdim=True)[0].expand_as(energy)-energy
        attention = self.softmax(energy_new)
        proj_value = x.view(m_batchsize, C, -1)

        out = torch.bmm(attention, proj_value)
        out = out.view(m_batchsize, C, height, width)

        out = self.gamma*out + x
        #out = F.avg_pool2d(out, out.size()[2:4])
        #out = self.last_conv(out)
        
        
        return out


In [ ]:
class RecNet(nn.Module):
    def __init__(self):
        super(RecNet, self).__init__()
        self.conv3d_1 = nn.Sequential(nn.Conv3d(1, 128, (1, 3, 3), 1), 
                        nn.BatchNorm3d(128),
                        nn.PReLU())
        
        self.conv3d_2 = nn.Sequential(nn.Conv3d(128, 64, (1, 3, 3), 1),
                        nn.BatchNorm3d(64),
                        nn.PReLU())
                        
        
        self.pool3d = nn.MaxPool3d((1, 1, 1), (1, 1, 1))
        
        self.deconv3d_1 = nn.Sequential(nn.ConvTranspose3d(64, 128, (1, 3, 3), 1),
                          nn.BatchNorm3d(128),
                          nn.PReLU())
        
        self.deconv3d_2 = nn.Sequential(nn.ConvTranspose3d(128, 1, (1, 3, 3), 1),
                          nn.BatchNorm3d(1))
        

    def forward(self, x):
        x = self.conv3d_1(x)
        x = self.conv3d_2(x)
        
        x = self.pool3d(x)
        
        x = self.deconv3d_1(x)
        x = self.deconv3d_2(x)
        
        
        return x.squeeze(1)

In [ ]:
class DANet(Module):
  def __init__(self):
    super(DANet,self).__init__()
    self.PAM_Module = PAM_Module(config.bands)
    self.CAM_Module = CAM_Module()
    self.RecNet = RecNet()
  def forward(self,x):
    
    P = self.PAM_Module(x)
    C = self.CAM_Module(x)
    #B,Ch,H,W = P.size()
    J = P + C
    J =  J.unsqueeze(1)
    ret = self.RecNet(J)
    
    
    
    return ret
    
    
danet_model = DANet().to(device)

In [ ]:

from torchsummary import summary
summary(danet_model,input_size=(config.bands,config.windowsize,config.windowsize))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1             [-1, 25, 7, 7]           5,025
            Conv2d-2             [-1, 25, 7, 7]           5,025
           Softmax-3               [-1, 49, 49]               0
            Conv2d-4            [-1, 200, 7, 7]          40,200
        PAM_Module-5            [-1, 200, 7, 7]               0
           Softmax-6             [-1, 200, 200]               0
        CAM_Module-7            [-1, 200, 7, 7]               0
            Conv3d-8       [-1, 128, 200, 5, 5]           1,280
       BatchNorm3d-9       [-1, 128, 200, 5, 5]             256
            PReLU-10       [-1, 128, 200, 5, 5]               1
           Conv3d-11        [-1, 64, 200, 3, 3]          73,792
      BatchNorm3d-12        [-1, 64, 200, 3, 3]             128
            PReLU-13        [-1, 64, 200, 3, 3]               1
        MaxPool3d-14        [-1, 64, 20

In [ ]:

optimizer = optim.SGD(danet_model.parameters(), lr=config.learning_rate, momentum=config.momentum)

In [ ]:
top = config.top

In [ ]:


import skimage
import kornia
global bsnlist
ssim = kornia.losses.SSIM(5, reduction='none')
psnr = kornia.losses.PSNRLoss(2500)
from skimage import measure
ssim_list = []
psnr_list = []
l1_list = []
channel_weight_list = []
def train(epoch):    
    danet_model.train()
    ENTROPY = torch.zeros(config.bands)
    
    for batch_idx, (data, __) in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        output = danet_model(data)
        loss = F.l1_loss(output,data)
        loss.backward()
        optimizer.step()
        D = output.detach().cpu().numpy()
        for i in range(0,config.bands):

          ENTROPY[i]+=skimage.measure.shannon_entropy(D[:,i,:,:])
        
        if batch_idx % (0.5*len(train_loader)) == 0:



            L1 = loss.item()
            wandb.log({"loss":L1})
            l1_list.append(L1)
            ssim_val = torch.mean(ssim(data,output))
            wandb.log({"SSIM":1.-ssim_val})
            ssim_list.append(1.-ssim_val)
            psnr_val = psnr(data,output)
            wandb.log({"PSNR":psnr_val})
            psnr_list.append(psnr_val)
        
        
    ENTROPY = np.array(ENTROPY)
    bsnlist = np.asarray(ENTROPY.argsort()[-top:][::-1])
    print('Top {} bands with Entropy ->'.format(top),list(bsnlist))
    
    

wandb.watch(danet_model)
for epoch in range(0, config.epochs):
    train(epoch)
    

Top 25 bands with Entropy -> [103, 143, 144, 197, 1, 195, 145, 104, 89, 196, 198, 149, 52, 51, 31, 35, 119, 53, 49, 176, 125, 27, 156, 46, 137]
Top 25 bands with Entropy -> [104, 109, 102, 148, 143, 105, 195, 170, 175, 141, 194, 196, 176, 174, 157, 149, 173, 156, 103, 192, 197, 80, 136, 77, 2]
Top 25 bands with Entropy -> [167, 17, 113, 2, 46, 190, 69, 157, 18, 101, 141, 64, 92, 133, 173, 49, 59, 153, 21, 136, 77, 171, 44, 11, 75]
Top 25 bands with Entropy -> [113, 60, 24, 57, 169, 136, 34, 96, 46, 90, 188, 38, 118, 28, 87, 58, 77, 22, 54, 52, 2, 85, 63, 17, 20]
Top 25 bands with Entropy -> [11, 22, 47, 99, 92, 8, 26, 29, 15, 12, 13, 42, 61, 79, 135, 174, 33, 30, 86, 81, 66, 37, 53, 21, 157]
Top 25 bands with Entropy -> [162, 131, 50, 132, 113, 29, 92, 83, 125, 172, 99, 7, 58, 66, 45, 27, 22, 129, 114, 54, 56, 166, 20, 72, 23]
Top 25 bands with Entropy -> [11, 72, 93, 80, 52, 19, 157, 35, 43, 29, 56, 55, 67, 161, 131, 84, 86, 90, 109, 117, 22, 36, 32, 137, 59]
Top 25 bands with Entropy